## DataBase  creator
This is an example what we want to do, from a text we generate the json file

In [ ]:
text='''
NO.1  A Machine Learning Specialist is working with multiple data sources containing billions of
records that need to be joined. What feature engineering and model development approach should
the Specialist take with a dataset this large?
A. Use an Amazon SageMaker notebook for both feature engineering and model development
B. Use an Amazon SageMaker notebook for feature engineering and Amazon ML for model
development
C. Use Amazon EMR for feature engineering and Amazon SageMaker SDK for model development
D. Use Amazon ML for both feature engineering and model development.
Answer:  C
Explanation:
Amazon EMR is a service that can process large amounts of data efficiently and cost-effectively. It can
run distributed frameworks such as Apache Spark, which can perform feature engineering on big
data. Amazon SageMaker SDK is a Python library that can interact with Amazon SageMaker service to
train and deploy machine learning models. It can also use Amazon EMR as a data source for training
data. References:
Amazon EMR
Amazon SageMaker SDK
NO.2  A Machine Learning Specialist is building a supervised model that will evaluate customers'
satisfaction with their mobile phone service based on recent usage The model's output should infer
whether or not a customer is likely to switch to a competitor in the next 30 days Which of the
following modeling techniques should the Specialist use1?
A. Time-series prediction
B. Anomaly detection
C. Binary classification
D. Regression
Answer:  C
Explanation:
The modeling technique that the Machine Learning Specialist should use is binary classification.
Binary classification is a type of supervised learning that predicts whether an input belongs to one of
two possible classes. In this case, the input is the customer's recent usage data and the output is
whether or not the customer is likely to switch to a competitor in the next 30 days. This is a binary
outcome, either yes or no, so binary classification is suitable for this problem. The other options are
not appropriate for this problem. Time-series prediction is a type of supervised learning that
forecasts future values based on past and present data. Anomaly detection is a type of unsupervised
learning that identifies outliers or abnormal patterns in the data. Regression is a type of supervised
learning that estimates a continuous numerical value based on the input features. References: Binary
Classification, Time Series Prediction, Anomaly Detection, Regression

'''
questions=[
    {
        'question': 'A Machine Learning Specialist is working with multiple data sources containing billions of records that need to be joined. What feature engineering and model development approach should the Specialist take with a dataset this large?',
        'options': [
            'A. Use an Amazon SageMaker notebook for both feature engineering and model development',
            'B. Use an Amazon SageMaker notebook for feature engineering and Amazon ML for model development',
            'C. Use Amazon EMR for feature engineering and Amazon SageMaker SDK for model development',
            'D. Use Amazon ML for both feature engineering and model development.'
        ],
        'correct': 'C. Use Amazon EMR for feature engineering and Amazon SageMaker SDK for model development',
        'explanation': 'The modeling technique that the Machine Learning Specialist should use is not applicable in this case, as the question is about feature engineering and model development approach, not about modeling technique. The correct answer is based on the large size of the dataset.',
        'references': 'Amazon SageMaker, Amazon EMR, Amazon ML'
    },
    {
        'question': 'A Machine Learning Specialist is building a supervised model that will evaluate customers satisfaction with their mobile phone service based on recent usage. The model\'s output should infer whether or not a customer is likely to switch to a competitor in the next 30 days. Which of the following modeling techniques should the Specialist use?',
        'options': [
            'A. Time-series prediction',
            'B. Anomaly detection',
            'C. Binary classification',
            'D. Regression'
        ],
        'correct': 'C. Binary classification',
        'explanation': 'The modeling technique that the Machine Learning Specialist should use is binary classification. Binary classification is a type of supervised learning that predicts whether an input belongs to one of two possible classes. In this case, the input is the customer\'s recent usage data and the output is whether or not the customer is likely to switch to a competitor in the next 30 days. This is a binary outcome, either yes or no, so binary classification is suitable for this problem.',
        'references': 'Binary Classification, Time Series Prediction, Anomaly Detection, Regression'
    }
]

In [188]:

import PyPDF2
import os
# Get the current working directory
current_dir = os.getcwd()
# Define the file path by appending the file name to the current directory
pdf_file_path = os.path.join(current_dir, 'exams', 'AWS', 'MLS-C01.pdf')

def pdf_to_text(pdf_file_path):
    # Open the PDF file in read-binary mode
    with open(pdf_file_path, 'rb') as f:
        # Create a PyPDF2 reader object
        pdf = PyPDF2.PdfReader(f)
        
        # Initialize an empty string to store the text
        text = ''
        
        # Iterate through all the pages in the PDF
        for page in pdf.pages:
            # Extract the text from the page and append it to the text string
            text += page.extract_text()
        
        # Return the extracted text
        return text

# Call the pdf_to_text function and print the result
text = pdf_to_text(pdf_file_path)
#print(text)


In [190]:
def extract_questions_v1(text):
    questions = []
    current_question = None
    in_explanation = False
    for line in text.split('\n'):
        line = line.strip()

        if re.match(r'NO\.\d+', line):
            if current_question:
                questions.append(current_question)
            current_question = {
                'question': '',
                'options': [],
                'correct': '',
                'explanation': '',
            }
            current_question['question'] = re.sub(r'NO\.\d+', '', line).strip()
            in_explanation = False
        elif re.match(r'[A-Z]\.', line):
            current_question['options'].append(line.strip())
        elif line.startswith('Answer:'):
            current_question['correct'] = line.replace('Answer:', '').strip()
        elif line.startswith('Explanation:'):
            in_explanation = True
            current_question['explanation'] = line.replace('Explanation:', '').strip()
        elif line.startswith('References:'):
            in_explanation = False
            current_question['references'] = line.replace('References:', '').strip()
        elif in_explanation:
            current_question['explanation'] += ' ' + line
    if current_question:
        questions.append(current_question)
    return questions
# Extract questions and format as JSON
questions = extract_questions_v1(text)
json_questions = json.dumps(questions, indent=4)
#print(json_questions)

In [191]:
import re
import json

def extract_questions(text):
    questions = []
    
    # Split text into individual question blocks
    question_blocks = re.split(r'NO\.\d+', text)
    question_blocks = [block.strip() for block in question_blocks if block.strip()]

    for block in question_blocks:
        # Split the block into question part, options, answer, explanation, and references
        question_part, *rest = re.split(r'(?:[A-D]\. )', block)
        options_part = ' '.join(rest).strip()
        
        # Extracting options
        options = re.findall(r'[A-D]\. [^\n]+', block)
        
        # Extracting the answer and explanation
        answer_match = re.search(r'Answer:\s*([A-D])', block)
        explanation_match = re.search(r'Explanation:\s*(.*?)References:', block, re.DOTALL)
        references_match = re.search(r'References:\s*(.*)', block, re.DOTALL)

        if answer_match and explanation_match:
            correct_letter = answer_match.group(1)
            explanation = explanation_match.group(1).strip()
            references = references_match.group(1).strip() if references_match else ""

            # Get correct option text
            correct_index = ord(correct_letter) - ord('A')
            if 0 <= correct_index < len(options):
                correct_option = options[correct_index]
            else:
                correct_option = correct_letter

            question = {
                'question': question_part.strip(),
                'options': options,
                'correct': correct_option,
                'explanation': explanation,
                'references': references
            }
            
            questions.append(question)

    return questions


In [192]:
questions = extract_questions(text)
print(json.dumps(questions[:1], indent=4))

[
    {
        "question": "A Machine Learning Specialist is working with multiple data sources containing billions of\nrecords that need to be joined. What feature engineering and model development approach should\nthe Specialist take with a dataset this large?",
        "options": [
            "A. Use an Amazon SageMaker notebook for both feature engineering and model development",
            "B. Use an Amazon SageMaker notebook for feature engineering and Amazon ML for model",
            "C. Use Amazon EMR for feature engineering and Amazon SageMaker SDK for model development",
            "D. Use Amazon ML for both feature engineering and model development."
        ],
        "correct": "C. Use Amazon EMR for feature engineering and Amazon SageMaker SDK for model development",
        "explanation": "Amazon EMR is a service that can process large amounts of data efficiently and cost-effectively. It can\nrun distributed frameworks such as Apache Spark, which can perform feature

In [193]:
len(questions)

27

In [205]:
import os
import json
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, 'exams', 'AWS')
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
file_path = os.path.join(folder_path, 'MLS-C01.json')

with open(file_path, 'w') as f:
    json.dump(questions, f, indent=4)

In [223]:
def select_exam_(exam_name):
    current_dir = os.getcwd()
    folder_path = os.path.join(current_dir, 'exams', 'AWS')
    file_path = os.path.join(folder_path, f'{exam_name}.json')

    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            questions = json.load(f)
        print("Loaded {} questions".format(len(questions)))    
        return questions
    else:
        return None

In [224]:
selected_questions = select_exam_('MLS-C01')

Loaded 27 questions


In [225]:
print(json.dumps(questions[:1], indent=4))

[
    {
        "question": "A Machine Learning Specialist is working with multiple data sources containing billions of\nrecords that need to be joined. What feature engineering and model development approach should\nthe Specialist take with a dataset this large?",
        "options": [
            "A. Use an Amazon SageMaker notebook for both feature engineering and model development",
            "B. Use an Amazon SageMaker notebook for feature engineering and Amazon ML for model",
            "C. Use Amazon EMR for feature engineering and Amazon SageMaker SDK for model development",
            "D. Use Amazon ML for both feature engineering and model development."
        ],
        "correct": "C. Use Amazon EMR for feature engineering and Amazon SageMaker SDK for model development",
        "explanation": "Amazon EMR is a service that can process large amounts of data efficiently and cost-effectively. It can\nrun distributed frameworks such as Apache Spark, which can perform feature

In [226]:

import os
import json

# Function to load question sets from a directory
def load_question_sets(directory='questions'):
    question_sets = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            question_sets.append(filename[:-5])  # remove the .json extension
    return question_sets
exams = load_question_sets('exams/AWS')
print("question_sets:", exams)


question_sets: ['MLS-C01']


In [227]:
selected_questions = select_exam_('MLS-C01')


Loaded 27 questions


In [229]:
selected_questions[0]

{'question': 'A Machine Learning Specialist is working with multiple data sources containing billions of\nrecords that need to be joined. What feature engineering and model development approach should\nthe Specialist take with a dataset this large?',
 'options': ['A. Use an Amazon SageMaker notebook for both feature engineering and model development',
  'B. Use an Amazon SageMaker notebook for feature engineering and Amazon ML for model',
  'C. Use Amazon EMR for feature engineering and Amazon SageMaker SDK for model development',
  'D. Use Amazon ML for both feature engineering and model development.'],
 'correct': 'C. Use Amazon EMR for feature engineering and Amazon SageMaker SDK for model development',
 'explanation': 'Amazon EMR is a service that can process large amounts of data efficiently and cost-effectively. It can\nrun distributed frameworks such as Apache Spark, which can perform feature engineering on big\ndata. Amazon SageMaker SDK is a Python library that can interact wi